# Paso 1: Importar Librerías de Python


In [ ]:
#pip install scikit-surprise

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from google.colab import drive

from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout, Dot
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Paso 2: Lectura del Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')

<ipython-input-5-1f8e3361d82a>:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/EspecializacionA&DS/Monografia/2doSemestre/DataFinal_Amazon.csv')  # Descomenta y proporciona la ruta si estás cargando un archivo CSV


In [ ]:
# Eliminemos posibles duplicados:
df = df.drop_duplicates()

In [ ]:
df.shape

(4641903, 10)

# Modelo de Filtrado Colaborativo usando SVD

Utilizaremos SVD para hacer recomendaciones basadas en interacciones pasadas.


Funcionamiento:

SVD es una técnica matemática que descompone una matriz en tres matrices más pequeñas: U, Σ y V*. En el contexto de sistemas de recomendación, se usa para factorizar la matriz de usuario-ítem en componentes latentes, capturando patrones subyacentes en los datos.

Ventajas:

Es matemáticamente robusto y ha sido una técnica establecida durante mucho tiempo.

Puede capturar relaciones no evidentes en los datos.

Reducción de dimensionalidad: al capturar la esencia de los datos en factores latentes, se puede trabajar con dimensiones reducidas.

Desventajas:

No maneja bien datos faltantes. La matriz de usuario-ítem suele ser dispersa, y el SVD estándar no se diseñó para manejar matrices con muchos valores faltantes.

Puede ser computacionalmente costoso para matrices muy grandes.

In [ ]:
# Leer y procesar datos
reader = Reader()
data = Dataset.load_from_df(df[['reviewerID', 'asin', 'overall']], reader)

# Dividir el dataset
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.3, random_state=10)

# Entrenar el modelo SVD
svd = SVD()
svd.fit(trainset)
predictions_svd = svd.test(testset)

# Función para calcular el Mean Absolute Percentage Error (MAPE)
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Evaluar el modelo
rmse_svd = accuracy.rmse(predictions_svd, verbose=True)
mse_svd = mean_squared_error([pred.r_ui for pred in predictions_svd], [pred.est for pred in predictions_svd])
mae_svd = mean_absolute_error([pred.r_ui for pred in predictions_svd], [pred.est for pred in predictions_svd])
mape_svd = mape([pred.r_ui for pred in predictions_svd], [pred.est for pred in predictions_svd])

print(f"SVD - RMSE: {rmse_svd}, MSE: {mse_svd}, MAE: {mae_svd}, MAPE: {mape_svd}")

RMSE: 1.0864
SVD - RMSE: 1.0864052122296506, MSE: 1.180276285159752, MAE: 0.8192398597949374, MAPE: 33.94576673547577


# Modelo de Filtrado Colaborativo usando Embedding con Keras:

Funcionamiento:

Los embeddings son representaciones vectoriales densas y de baja dimensión de ítems y/o usuarios. Estas representaciones capturan relaciones semánticas entre ítems o entre usuarios.

Un método popular para generar embeddings es la factorización de matrices, como la descomposición en valores singulares (SVD). En el contexto de sistemas de recomendación, se busca factorizar la matriz de interacciones usuario-ítem en dos matrices más pequeñas (una para los usuarios y otra para los ítems) cuyo producto aproximado reproduce la matriz original lo mejor posible.

Una vez que se han obtenido los embeddings, la predicción de una calificación o interacción entre un usuario e ítem se realiza tomando el producto escalar entre sus embeddings respectivos.

Ventajas:

Simplicidad y eficiencia en términos computacionales.

Puede manejar grandes conjuntos de datos debido a su naturaleza de baja dimensión.

Es efectivo para capturar patrones subyacentes en los datos.

Desventajas:

No tiene en cuenta características adicionales de usuarios o ítems.

Dificultades para manejar nuevos ítems o usuarios (problema de arranque en frío).

In [ ]:
data = df

In [ ]:
# Crear un LabelEncoder para cada columna
reviewerID_encoder = LabelEncoder()
asin_encoder = LabelEncoder()

# Ajustar y transformar las columnas
data['reviewerID_encoded'] = reviewerID_encoder.fit_transform(data['reviewerID'])
data['asin_encoded'] = asin_encoder.fit_transform(data['asin'])

# Mostrar las primeras filas del DataFrame para verificar
#print(data[['reviewerID', 'reviewerID_encoded', 'asin', 'asin_encoded']].head())


# Número de usuarios únicos y número de ítems únicos
n_users = len(np.unique(data['reviewerID_encoded']))
n_items = len(np.unique(data['asin_encoded']))

# Dimensiones del embedding
embedding_dim = 10

# Entradas
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

# Embeddings
user_embedding = Embedding(n_users, embedding_dim)(user_input)
item_embedding = Embedding(n_items, embedding_dim)(item_input)

# Producto punto para predecir la valoración/rating
merged = Dot(axes=2)([user_embedding, item_embedding])
merged = Flatten()(merged)

# Modelo
model = Model(inputs=[user_input, item_input], outputs=merged)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

verbose=0: No muestra ninguna barra de progreso ni métricas.

verbose=1: Muestra una barra de progreso y actualiza las métricas después de cada lote.

verbose=2: Muestra las métricas después de cada época, pero no muestra la barra de progreso.

In [ ]:
from sklearn.model_selection import train_test_split

# Crear los arrays con los datos
user_ids = data['reviewerID_encoded'].values
item_ids = data['asin_encoded'].values
ratings = data['overall'].values

# Dividir en train y test
(user_ids_train, user_ids_test, item_ids_train, item_ids_test, ratings_train, ratings_test) = train_test_split(
    user_ids, item_ids, ratings,test_size=0.3,random_state=42)

In [ ]:
# Entrenar el modelo con el conjunto de entrenamiento
model.fit([user_ids_train, item_ids_train], ratings_train, epochs=10, batch_size=80000, verbose=1)

predictions = model.predict([user_ids_test, item_ids_test])

Epoch 1/10
41/41 [==============================] - 11s 231ms/step - loss: 19.4976 - mae: 4.2587
Epoch 2/10
41/41 [==============================] - 8s 201ms/step - loss: 19.4897 - mae: 4.2578
Epoch 3/10
41/41 [==============================] - 7s 180ms/step - loss: 19.4762 - mae: 4.2563
Epoch 4/10
41/41 [==============================] - 8s 189ms/step - loss: 19.4483 - mae: 4.2532
Epoch 5/10
41/41 [==============================] - 8s 192ms/step - loss: 19.3959 - mae: 4.2472
Epoch 6/10
41/41 [==============================] - 6s 144ms/step - loss: 19.3089 - mae: 4.2373
Epoch 7/10
41/41 [==============================] - 5s 113ms/step - loss: 19.1793 - mae: 4.2224
Epoch 8/10
41/41 [==============================] - 5s 119ms/step - loss: 19.0024 - mae: 4.2017
Epoch 9/10
41/41 [==============================] - 5s 108ms/step - loss: 18.7768 - mae: 4.1750
Epoch 10/10
41/41 [==============================] - 4s 94ms/step - loss: 18.5035 - mae: 4.1422


In [ ]:
# Evaluar el modelo con el conjunto de prueba
loss, mae = model.evaluate([user_ids_test, item_ids_test], ratings_test, batch_size=80000)
print(f'Test Loss: {loss:.4f}')
print(f'Test MAE: {mae:.4f}')

18/18 [==============================] - 0s 2ms/step - loss: 19.2073 - mae: 4.2225
Test Loss: 19.2073
Test MAE: 4.2225


In [ ]:
# MSE
mse = mean_squared_error(ratings_test, predictions)
print(f"MSE: {mse:.4f}")

# RMSE
rmse = np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

# MAE
mae = mean_absolute_error(ratings_test, predictions)
print(f"MAE: {mae:.4f}")

# MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((ratings_test - predictions.flatten()) / ratings_test)) * 100
print(f"MAPE: {mape:.2f}%")

MSE: 19.2073
RMSE: 4.3826
MAE: 4.2225
MAPE: 99.04%


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa con Keras:

Funcionamiento:

Estos sistemas toman características de los ítems y/o usuarios y las pasan a través de una o varias capas de neuronas para obtener una predicción.
Las redes neuronales son capaces de capturar interacciones no lineales entre características, lo que las hace poderosas para tareas de modelado complejas.

Ventajas:

Capacidad de modelar relaciones no lineales.

Flexibilidad para incorporar múltiples fuentes de datos o características.

Puede manejar arranques en frío al incorporar características de nuevos ítems o usuarios.

Desventajas:

Mayor costo computacional en comparación con los sistemas basados únicamente en embeddings.

Riesgo de sobreajuste si no se tiene un conjunto de datos lo suficientemente grande.

In [ ]:
# Convertir reviewerID y asin a índices numéricos secuenciales
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
hidden_units = [128, 64, 32]  # Unidades en las capas ocultas
dropout_rate = 0.2

# Arquitectura del modelo
user_input = Input(shape=[1], name='user_input')
item_input = Input(shape=[1], name='item_input')
concat = Concatenate()([user_input, item_input])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu')(dense)
    dense = Dropout(dropout_rate)(dense)
output = Dense(1)(dense)
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

In [ ]:
# Entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# Usando validation_split, divide los datos
from sklearn.model_selection import train_test_split

train_size = int(0.7 * len(user_data))
user_data_train, user_data_val = user_data[:train_size], user_data[train_size:]
item_data_train, item_data_val = item_data[:train_size], item_data[train_size:]
y_true = rating_data[train_size:]

history = model.fit([user_data_train, item_data_train], rating_data[:train_size],
                    epochs=10, validation_data=([user_data_val, item_data_val], y_true),
                    batch_size=80000, verbose=1)

# Predicciones
predictions = model.predict([user_data_val, item_data_val])

Epoch 1/10
41/41 [==============================] - 3s 18ms/step - loss: 1135308928.0000 - val_loss: 14672278.0000
Epoch 2/10
41/41 [==============================] - 1s 13ms/step - loss: 73755912.0000 - val_loss: 782145.6875
Epoch 3/10
41/41 [==============================] - 1s 13ms/step - loss: 12576571.0000 - val_loss: 34165.4375
Epoch 4/10
41/41 [==============================] - 1s 13ms/step - loss: 3624744.7500 - val_loss: 6065.4355
Epoch 5/10
41/41 [==============================] - 1s 13ms/step - loss: 1757924.8750 - val_loss: 1307.5425
Epoch 6/10
41/41 [==============================] - 1s 13ms/step - loss: 1131483.8750 - val_loss: 477.2494
Epoch 7/10
41/41 [==============================] - 1s 13ms/step - loss: 835641.3125 - val_loss: 290.3395
Epoch 8/10
41/41 [==============================] - 1s 13ms/step - loss: 650706.6875 - val_loss: 168.8172
Epoch 9/10
41/41 [==============================] - 1s 13ms/step - loss: 521061.7188 - val_loss: 99.4635
Epoch 10/10
43518/43518 

In [ ]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# Funciones MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 81.00335522250023
RMSE: 9.000186399319752
MAE: 8.57458161361687
MAPE: 223.03449762080908%


# Modelo de Filtrado Colaborativo usando Red Neuronal Multicapa y Embedding con Keras:

Este enfoque, que combina embeddings y redes neuronales multicapa, puede capturar interacciones más complejas y no lineales entre usuarios y artículos. Sin embargo, es crucial prestar atención al sobreajuste y asegurarse de que el modelo no esté simplemente memorizando los datos. Por lo tanto, es recomendable emplear técnicas de regularización, ajustar hiperparámetros y validar el rendimiento con un conjunto de datos de validación.

Funcionamiento:

Combina lo mejor de ambos mundos. Primero, se utilizan embeddings para convertir ítems y usuarios en representaciones vectoriales densas. Luego, estas representaciones se pasan a través de una red neuronal para hacer la predicción.

El proceso generalmente comienza con capas de embedding que convierten identificadores de usuarios e ítems en vectores. Estos vectores luego se pasan a través de capas densas para obtener la predicción final.

Ventajas:

Capacidad de capturar patrones subyacentes en los datos mediante embeddings y modelar interacciones no lineales mediante la red neuronal.

Flexibilidad para incorporar características adicionales.

Potencialmente más preciso que cualquiera de los otros dos métodos por separado.

Desventajas:

Mayor complejidad en el modelado y entrenamiento.

Mayor costo computacional.

Requiere un ajuste más cuidadoso y riesgo de sobreajuste si no se gestiona adecuadamente.

In [ ]:
# Convertir reviewerID y asin a índices numéricos secuenciales
df['user_id'] = df['reviewerID'].astype('category').cat.codes.values
df['item_id'] = df['asin'].astype('category').cat.codes.values

# Número de usuarios e ítems
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()

# Hiperparámetros
n_latent_factors = 50  # Número de factores latentes
hidden_units = [128, 64]  # Unidades en las capas ocultas
dropout_rate = 0.2

# Arquitectura del modelo
user_input = Input(shape=[1], name='user_input')
item_input = Input(shape=[1], name='item_input')
user_embedding = Embedding(n_users, n_latent_factors, name='user_embedding')(user_input)
item_embedding = Embedding(n_items, n_latent_factors, name='item_embedding')(item_input)
user_vec = Flatten(name='flatten_users')(user_embedding)
item_vec = Flatten(name='flatten_items')(item_embedding)
concat = Concatenate()([user_vec, item_vec])
dense = concat
for units in hidden_units:
    dense = Dense(units, activation='relu')(dense)
    dense = Dropout(dropout_rate)(dense)
output = Dense(1)(dense)
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

In [ ]:
# Entrenamiento
user_data = df['user_id'].values
item_data = df['item_id'].values
rating_data = df['overall'].values

# Usando validation_split, divide los datos
train_size = int(0.7 * len(user_data))
user_data_train, user_data_val = user_data[:train_size], user_data[train_size:]
item_data_train, item_data_val = item_data[:train_size], item_data[train_size:]
y_true = rating_data[train_size:]

history = model.fit([user_data_train, item_data_train], rating_data[:train_size],
                    epochs=10, validation_data=([user_data_val, item_data_val], y_true),
                    batch_size=80000, verbose=1)

# Predicciones
predictions = model.predict([user_data_val, item_data_val])

Epoch 1/10
41/41 [==============================] - 13s 290ms/step - loss: 13.9407 - val_loss: 8.0267
Epoch 2/10
41/41 [==============================] - 11s 256ms/step - loss: 1.8954 - val_loss: 5.2937
Epoch 3/10
41/41 [==============================] - 9s 222ms/step - loss: 1.1846 - val_loss: 4.9302
Epoch 4/10
41/41 [==============================] - 9s 219ms/step - loss: 0.9862 - val_loss: 4.7563
Epoch 5/10
41/41 [==============================] - 9s 232ms/step - loss: 0.9110 - val_loss: 4.6494
Epoch 6/10
41/41 [==============================] - 7s 166ms/step - loss: 0.8733 - val_loss: 4.5646
Epoch 7/10
41/41 [==============================] - 7s 177ms/step - loss: 0.8489 - val_loss: 4.4939
Epoch 8/10
41/41 [==============================] - 6s 149ms/step - loss: 0.8289 - val_loss: 4.4362
Epoch 9/10
41/41 [==============================] - 6s 147ms/step - loss: 0.8122 - val_loss: 4.3828
Epoch 10/10
43518/43518 [==============================] - 60s 1ms/step


In [ ]:
# MSE y RMSE
mse = mean_squared_error(y_true, predictions)
rmse = np.sqrt(mse)

# MAE
mae = mean_absolute_error(y_true, predictions)

# MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_idx = y_true != 0  # Para evitar divisiones por cero
    return np.mean(np.abs((y_true[non_zero_idx] - y_pred[non_zero_idx]) / y_true[non_zero_idx])) * 100

def compute_mape_by_batch(y_true, predictions, batch_size=50000):
    mape_sum = 0
    num_batches = int(np.ceil(len(y_true) / batch_size))

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        batch_y_true = y_true[start_idx:end_idx]
        batch_predictions = predictions[start_idx:end_idx]

        mape_sum += mean_absolute_percentage_error(batch_y_true, batch_predictions)

    return mape_sum / num_batches

# Luego llamas a la función
mape = compute_mape_by_batch(y_true, predictions)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MAPE: {mape}%')

MSE: 4.286940096338541
RMSE: 2.070492718252962
MAE: 1.8667582920725043
MAPE: 49.114634109069634%


Estas métricas te ayudarán a tener una mejor idea del desempeño de tu modelo. Por ejemplo:

MSE y RMSE son útiles cuando quieres penalizar grandes errores.

MAE te da una idea del error medio sin considerar la dirección del error.

MAPE es útil cuando quieres representar el error en términos porcentuales.

Para una evaluación completa, es recomendable utilizar un conjunto de validación aparte (es decir, no solo depender del validation_split). Esto asegura que estás evaluando el desempeño en datos que el modelo nunca ha visto durante el entrenamiento.